In [87]:
from dash import Dash, html, dcc
from dash.dependencies import Output, Input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import os
import glob


In [12]:
def read_files(folder_path, file_extension):
    # Create a file path pattern using glob
    pattern = os.path.join(folder_path, f"*.{file_extension}")
    filenames = []

    # Use glob to get a list of files matching the pattern
    files = glob.glob(pattern)
    for file in files:
        filenames.append(file.split("\\")[1])

    # Return the count of matching files
    return filenames

# Example usage
folder_path = "C:/Users/user/Documents/GitHub/Automotive_Diagnostics/OBD-II-Dataset"
file_extension = "csv"

data_files = read_files(folder_path, file_extension)
# print(data_files)

In [90]:
app = Dash(__name__)
origin = "Nigeria"
destination = "UK"
cond = "Normal"
df = pd.read_csv("OBD-II-Dataset/2017-07-05_Seat_Leon_RT_S_Stau.csv")
df = df.iloc[10:]
df["Time"] = pd.to_datetime(df["Time"])
df = df.set_index("Time")
df.columns = ['Engine_Coolant_Temperature', 'Intake_Manifold_Abs_Pressure', 'Engine_RPM', 'Vehicle_Speed', 'Intake_Air_Temperature', 'AirFlow_Rate', 'Throttle_Position', 'Air_Temperature', 'Acc_Pedal_Pos_D', 'Acc_Pedal_Pos_E']


app.layout = html.Div([
    html.H1("Automotive Diagnostic Dashboard"),
    "Select a data to visualise:!",
    dcc.Dropdown(
        id="file_name",
        options=data_files,
        value=""
    ),
    html.Div([
    # html.P(children=["...text", num]) 
    html.Span(id='origin_loc', children=['Origin: ', origin], style={'flex': '1'}),
    html.Span(id='destination_loc', children=['Destination: ', destination], style={'flex': '1', 'text-align': 'center'}),
    html.Span(id='cond', children=['Condition: ', cond], style={'align-self': '1', 'text-align': 'right'})
    ],style={'display': 'flex', 'justify-content': 'space-between'}),
    html.H3("Assessing Engine Load"),
    # dcc.Graph(
    #     id="visual",
    #     figure=px.scatter(df, x=df['AirFlow_Rate'], y=df['Engine_RPM'])
    # ),
    html.Div([generate_subplot(column) for column in df.columns])
])

# def produce_plot(filename, x-axis, y-axis):
#         plt.figure(figsize=(10,6))
        
# Define a function to generate subplots
def generate_subplot(column_name):
    return dcc.Graph(
        id=column_name,
        figure={
            'data': [
                go.Scatter(
                    x=df['x'],
                    y=df[column_name],
                    mode='lines+markers',
                    name=column_name
                )
            ],
            'layout': go.Layout(
                title=column_name,
                xaxis={'title': 'X'},
                yaxis={'title': 'Y'},
                height=200,
                width=400
            )
        }
    )


def get_data(file_name):
    data = pd.read_csv("OBD-II-Dataset/" + file_name)
    data = data.iloc[10:]
    data["Time"] = pd.to_datetime(data["Time"])
    data = data.set_index("Time")
    data.columns = ['Engine_Coolant_Temperature', 'Intake_Manifold_Abs_Pressure', 'Engine_RPM', 'Vehicle_Speed', 'Intake_Air_Temperature', 'AirFlow_Rate', 'Throttle_Position', 'Air_Temperature', 'Acc_Pedal_Pos_D', 'Acc_Pedal_Pos_E']
    return data

@app.callback([Output("origin_loc", "children"), Output("destination_loc", "children"), Output("cond", "children")], Input("file_name", "value"))
def update_data_desc(fname):
    df = get_data(fname)
    origin_code = {"KA":"Karlsruhe", "RT": "Reutlingen", "S":"Stuttgart", "BB":"Boblingen"}
    dest_code = {"CW": "Calw", "KA":"Karlsruhe", "RT": "Reutlingen", "S":"Stuttgart", "BB":"Boblingen"}
    cond_code = {"Stau": "Busy", "Normal": "Normal", "Frei": "Free"}
    code_list = fname.split("_")
    if len(fname) > 1:        
        origin = origin_code[code_list[3]]
        destination = origin_code[code_list[4]]
        if "Stau" in fname:
            cond = "Busy"
        elif "Normal" in fname:
            cond = "Normal"
        elif "Frei" in fname:
            cond = "Free"
    # return f"File selected: {fname}"
    return ['Origin: ' + origin, ' Destination: ' + destination, ' Condition: ' + cond]

if __name__ == "__main__":
    app.run_server(mode="inline", port=8051)

NameError: name 'generate_subplot' is not defined